In [1]:
import cv2 as cv
import numpy as np
import math
import time
import heapdict

In [2]:
class cordi:
    def __init__(self, i , j):
        self.i = i
        self.j = j
        
    def __eq__(self, other):
         return self.i == other.i and self.j == other.j
        
    def __hash__(self):
        return hash((self.i, self.j))

class val:
    def __init__(self, f, g):
        self.f = f
        self.g = g
        
    def __eq__(self, other):
         return self.f == other.f and self.g == other.g
        
class olm:
    def __init__(self, cordi , val):
        self.val = val
        self.cordi = cordi

In [3]:
def valid(mat, cordi):
    row = cordi.i
    col = cordi.j
    if (row >= 0) and (row < 100) and (col >= 0) and (col < 100) and ((mat[row][col] == np.array([0,0,0])).all() or (mat[row][col]==[ 60, 76, 231]).all() or (mat[row][col]==[ 255, 0, 0]).all()):
        return (True)
    else:
        return (False)

In [4]:
def hCalc(cordi, dest, types):

    if types == "Dijkstra":
        return 0.0
    
    if types == "Admissible":
        return((1.00005*(math.sqrt((cordi.i - dest.i) * (cordi.i - dest.i) + (cordi.j - dest.j) * (cordi.j - dest.j))))-1)
        
    if types == "Non-Admissible":
        return (10*math.sqrt((cordi.i - dest.i) * (cordi.i - dest.i) + (cordi.j - dest.j) * (cordi.j - dest.j)))
    
    if types == "Diagonal":
        return max(abs(cordi.i - dest.i),abs(cordi.j - dest.j))
    
    if types == "Manhattan":
        return (abs(cordi.i - dest.i) + abs(cordi.j - dest.j) )
    
    if types == "Euclid":
        return (math.sqrt((cordi.i - dest.i) * (cordi.i - dest.i) + (cordi.j - dest.j) * (cordi.j - dest.j)))
        

In [5]:
def trace_path(parent, dest, src, large):
    path = []

    i = dest.i
    j = dest.j
    
    while not(parent[i][j] == src):
        mat[i][j] = [203,192,255]

        out.write(large)
        
        tempi = parent[i][j].i
        tempj = parent[i][j].j
        
        i = tempi
        j= tempj
        
        for x in range(10):
            for y in range(10):
                large[10*i+x][10*j+y]=[255,255,0]


        path.append([i,j])
        
    path = path[::-1]
    return (len(path))
    

In [6]:
#main stuff
def aStar(mat, src, dest, types, case, large):
    
    parent = np.full((100,100), cordi(-1,-1))
    
    values = val(0,0)
    start = olm(src,values)
  
    openl = heapdict.heapdict()
    openl[start] = 0 
    
    record = {src:values}
    
    while (len(openl)!=0):
        
        leastF  = openl.popitem()[0]
        
        if (leastF.cordi==dest):
            trace_path(parent, dest, src, large) 
            return leastF.val.g 
        
        i = leastF.cordi.i
        j = leastF.cordi.j        
        
        if not(leastF.cordi==src):
            mat[i][j] = [0,165,255]
            for x in range(10):
                for y in range(10):
                    large[10*i+x][10*j+y]=[0,165,255]
                    
        out.write(large)

        
        for New in (cordi(i+1,j),cordi(i-1,j),cordi(i,j+1),cordi(i,j-1)):

            if valid(mat, New):
                g = leastF.val.g + 1.0 
                h = hCalc(New, dest, types) 
                f = g + h
                
                newVal = val(f,g)
                
                oldf = (record.get(New, val(14200,0))).f 
                
                if oldf > f:
                    parent[New.i][New.j] = leastF.cordi
                    openl[olm(New, newVal)] = f
                    
                    if not(New==src or New==dest):
                        mat[New.i][New.j] = [255,0,0]
    
                        for x in range(10):
                            for y in range(10):
                                large[10*New.i+x][10*New.j+y]=[255,0,0]

                    record.update ({New:newVal})
            
            
        if case==2:
            for New in (cordi(i+1,j+1),cordi(i-1,j-1),cordi(i-1,j+1),cordi(i+1,j-1)):

                if valid(mat, New):
                    g = leastF.val.g + 1.414213
                    h = hCalc(New, dest, types) 
                    f = g + h

                    newVal = val(f,g)

                    oldf = (record.get(New, val(14200,0))).f 

                    if oldf > f:
                        parent[New.i][New.j] = leastF.cordi
                        openl[olm(New, newVal)] = f
                        
                    if not(New==src or New==dest):
                        mat[New.i][New.j] = [255,0,0]
    
                        for x in range(10):
                            for y in range(10):
                                large[10*New.i+x][10*New.j+y]=[255,0,0]

                        record.update ({New:newVal})
            

    print("Failed to find the Destination Cell") 
    

    return 
    

In [7]:
for case in (1,2):
    print("Case:", str(case))
    print()
    
    for types in ("Dijkstra", "Admissible", "Non-Admissible", "Diagonal", "Manhattan", "Euclid"):
        
        img = cv.imread("Task_1_Low.png")
        mat = np.array(img) 
        
        filename = (str(case) + " " + types + ".mp4" )

        out = cv.VideoWriter(filename,cv.VideoWriter_fourcc(*'MP4V'), 500, (1000,1000))

        large = np.full((1000,1000,3),int(1))
        large = large.astype(np.uint8)

        for row in range(1000):
            for col in range(1000):
                for z in range(3):
                    a=int(row//10)
                    b=int(col//10)
                    large[row][col][z]=mat[a][b][z]

        large = large.astype(np.uint8)

        out.write(large)
        
        print(types + ":")
        img = cv.imread("Task_1_Low.png")
        mat = np.array(img) 
        
        for row in range(100):
            for col in range(100):
                if ((mat[row,col] == np.array([113, 204, 45])).all()):
                    src = cordi(row,col)
                if ((mat[row,col] == np.array([60, 76, 231])).all()):
                    dest = cordi(row,col)

        print("Path Length:",round(aStar(mat, src, dest, types, case, large),2))
        
        print()
        
        for x in range(3000):
            out.write(large)
        
        out.release()

    print()
    print()
    print()


Case: 1

Dijkstra:
Path Length: 181.0

Admissible:
Path Length: 181.0

Non-Admissible:
Path Length: 251.0

Diagonal:
Path Length: 181.0

Manhattan:
Path Length: 181.0

Euclid:
Path Length: 181.0




Case: 2

Dijkstra:
Path Length: 147.61

Admissible:
Path Length: 147.61

Non-Admissible:
Path Length: 157.89

Diagonal:
Path Length: 147.61

Manhattan:
Path Length: 147.61

Euclid:
Path Length: 147.61






### 